<a href="https://www.kaggle.com/code/marinabalakina/dll30-dz6-2-ipynb?scriptVersionId=156621214" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Задание 2.**

Выполнить практическую работу из лекционного ноутбука.

* Построить RNN-ячейку на основе полносвязных слоев
* Применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”

## 2.1. Импорт библиотек и пользовательские функции

In [4]:
import torch
import torch.nn as nn

from collections import Counter
import warnings

import string
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
%matplotlib inline
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
torch.autograd.set_detect_anomaly(True)
batch_size = 100
seq_size = 32
embedding_size = 64
lstm_size = 64
gradients_norm = 5
# GPU или CPU?
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
def doc2words(doc):
  # Это мешок слов
    words=[]
    for line in doc:
      words += line.split()
    return words

In [7]:
def removepunct(words):
  # Удаляем пунктуацию
    punct = set(string.punctuation)
    words = [''.join([char for char in list(word) if char not in punct]) for word in words]
    return words

In [8]:
def getvocab(words):
  # Словарь из мешка слов
    wordfreq = Counter(words)
    sorted_wordfreq = sorted(wordfreq, key=wordfreq.get)
    return sorted_wordfreq

In [9]:
def vocab_map(vocab):
    # 2 словаря - int to words and word to int
    int_to_vocab = {k:w for k,w in enumerate(vocab)}
    vocab_to_int = {w:k for k,w in int_to_vocab.items()}
    return int_to_vocab, vocab_to_int

In [10]:
def get_batches(words, vocab_to_int, batch_size, seq_size):
    # Генерируем батчи для  Xs и Ys: shape = (batchsize * num_batches) * seq_size
    word_ints = [vocab_to_int[word] for word in words]
    num_batches = int(len(word_ints) / (batch_size * seq_size))
    Xs = word_ints[:num_batches*batch_size*seq_size]
    Ys = np.zeros_like(Xs)
    Ys[:-1] = Xs[1:]
    Ys[-1] = Xs[0]
    Xs = np.reshape(Xs, (num_batches*batch_size, seq_size))
    Ys = np.reshape(Ys, (num_batches*batch_size, seq_size))
    
    # iterate over rows of Xs and Ys to generate batches
    for i in range(0, num_batches*batch_size, batch_size):
        yield Xs[i:i+batch_size, :], Ys[i:i+batch_size, :]

In [11]:
class RNNModule(nn.Module):
    # initialize RNN module
    def __init__(self, n_vocab, seq_size=32, embedding_size=64, lstm_size=64):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)
        
    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return logits, state
    
    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),torch.zeros(1, batch_size, self.lstm_size))

In [12]:
def get_loss_and_train_op(net, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    return criterion, optimizer

In [13]:
def generate_text(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()

    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))
    
    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])
    
    for _ in range(100):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    print(' '.join(words))

In [14]:
def train_rnn(words, vocab_to_int, int_to_vocab, n_vocab):
    
    # ячейка RNN
    net = RNNModule(n_vocab, seq_size, embedding_size, lstm_size)
    net = net.to(device)
    criterion, optimizer = get_loss_and_train_op(net, 0.01)

    iteration = 0
    
    # итерируемся по эпохам
    for e in tqdm(range(50)):
        # получаем батчи
        batches = get_batches(words, vocab_to_int, batch_size, seq_size)
        # инициализируем выход и сккрытое состояние
        state_h, state_c = net.zero_state(batch_size)

        # Передаем данные на GPU
        state_h = state_h.to(device)
        state_c = state_c.to(device)
        # итерируемся по батчам
        for x, y in tqdm(batches):
            iteration += 1

            # Переходим  в режим обучения
            net.train()

            # Обнуляем градиенты
            optimizer.zero_grad()

            # Передаем x и y на GPU
            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device)
            
            # Модель возвращает логиты, последнее скрытое состояние и новый выход
            logits, (state_h, state_c) = net(x, (state_h, state_c))
            loss = criterion(logits.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss_value = loss.item()

            # back-propagation
            loss.backward(retain_graph=True)

            _ = torch.nn.utils.clip_grad_norm_(net.parameters(), gradients_norm)
            
            # Обновляем параметры, выполняя шаг обучения
            optimizer.step()

            if iteration % 100 == 0:
                print('Epoch: {}/{}'.format(e, 200),'Iteration: {}'.format(iteration),'Loss: {}'.format(loss_value))

            # if iteration % 1000 == 0:
                # predict(device, net, flags.initial_words, n_vocab,vocab_to_int, int_to_vocab, top_k=5)
                # torch.save(net.state_dict(),'checkpoint_pt/model-{}.pth'.format(iteration))
                
    return net

## 2.2. Загрузка и предобработка данных

In [15]:
doc = pd.read_csv('/kaggle/input/the-simpsons-dataset/simpsons_script_lines.csv', usecols=['normalized_text'],low_memory=False)['normalized_text'].astype(str).to_list()

In [16]:
doc[:5]

['no actually it was a little of both sometimes when a disease is in all the magazines and all the news shows its only natural that you think you have it',
 'wheres mr bergstrom',
 'i dont know although id sure like to talk to him he didnt touch my lesson plan what did he teach you',
 'that life is worth living',
 'the polls will be open from now until the end of recess now just in case any of you have decided to put any thought into this well have our final statements martin']

In [17]:
# получаем мешок слов, удаляем пунктуацию
words = removepunct(doc2words(doc))
# Словарь из мешка слов
vocab = getvocab(words)
# 2 словаря - int_to_vocab и vocab_to_int
int_to_vocab, vocab_to_int = vocab_map(vocab)

## 2.3. Обучение модели

In [18]:
rnn_net = train_rnn(words, vocab_to_int, int_to_vocab, len(vocab))

  0%|          | 0/50 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Epoch: 0/200 Iteration: 100 Loss: 6.869081974029541
Epoch: 0/200 Iteration: 200 Loss: 6.405984401702881
Epoch: 0/200 Iteration: 300 Loss: 6.204434871673584
Epoch: 0/200 Iteration: 400 Loss: 6.468569278717041


0it [00:00, ?it/s]

Epoch: 1/200 Iteration: 500 Loss: 6.069814682006836
Epoch: 1/200 Iteration: 600 Loss: 5.84013557434082
Epoch: 1/200 Iteration: 700 Loss: 5.847909450531006
Epoch: 1/200 Iteration: 800 Loss: 5.760141372680664


...

Epoch: 47/200 Iteration: 19600 Loss: 4.582718849182129
Epoch: 47/200 Iteration: 19700 Loss: 4.7888383865356445
Epoch: 47/200 Iteration: 19800 Loss: 4.606385707855225
Epoch: 47/200 Iteration: 19900 Loss: 4.612706184387207


0it [00:00, ?it/s]

Epoch: 48/200 Iteration: 20000 Loss: 4.541013717651367
Epoch: 48/200 Iteration: 20100 Loss: 4.77940034866333
Epoch: 48/200 Iteration: 20200 Loss: 4.587284564971924
Epoch: 48/200 Iteration: 20300 Loss: 4.672327518463135


0it [00:00, ?it/s]

Epoch: 49/200 Iteration: 20400 Loss: 4.595296382904053
Epoch: 49/200 Iteration: 20500 Loss: 4.642980575561523
Epoch: 49/200 Iteration: 20600 Loss: 4.656783580780029
Epoch: 49/200 Iteration: 20700 Loss: 4.607407093048096
Epoch: 49/200 Iteration: 20800 Loss: 4.557277679443359


In [23]:
PATH = '/kaggle/working/rnn_model'
torch.save(rnn_net.state_dict(), PATH)

In [ ]:
# rnn_model.load_state_dict(torch.load(PATH))

## 2.4. Генерация реплик

In [19]:
generate_text(device, rnn_net, ['hey', 'you'], len(vocab), vocab_to_int, int_to_vocab)

hey you whatcha chong are a big deal well well it i want a man who said that was the only thing that you can see you in my arms out nan nan oh homer i think we got the lead and if we were raptured or do not have some other way out with the kids with the most heinous owner upon a coal of the state capitals first nan oh yeah yeah yeah i love you nan oh oh no you dont want a lot more about a pleasanttasting park highway embankments theres no way you want to do it oh
